In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [30]:
# Whether I am using the GPU or the CPU is checked in this block.
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
image_size = 784
hidden_size = 256
latent_size = 64

In [51]:
import torch.nn as nn

#Define a pretty simple Disriminator network which has:
# 1. Linear layer which takes in an input of image_size as 784 and hidden size 256
# 2. The output of the linear layer is passed through a ReLU activation function
# 3. The result of the activation function is then passed through a Linear layer which takes in 256 input and produces 256 outputs
# 4. This result is again passed through a leaky relu layer.
# 5. Finally the output is passed through a Linear layer and the result is passed through the sigmoid function.

D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [52]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

In [53]:
import sys
sys.path.append('/workspace/fl')
from process import preprocess as P

In [54]:
# Load MNIST Data
train_data, test_data = P.load_mnist_dataset()
batch_size = 100
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

In [61]:
#Training the discriminator

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

def train_discriminator(images):
    # Create the labels which are later used as input for the BCE loss
    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)
        
    # Loss for real images
    outputs = D(images)
    d_loss_real = criterion(outputs, real_labels)
    real_score = outputs

    # Loss for fake images
    z = torch.randn(batch_size, latent_size).to(device)
    fake_images = G(z)
    outputs = D(fake_images)
    d_loss_fake = criterion(outputs, fake_labels)
    fake_score = outputs
    
    d_loss = d_loss_real + d_loss_fake
    reset_grad()
    d_loss.backward()
    d_optimizer.step()
    
    return d_loss, real_score, fake_score

In [62]:
import os

sample_dir = './../data/samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [63]:
from IPython.display import Image
from torchvision.utils import save_image

sample_vectors = torch.randn(32, latent_size).to('cpu')

def save_fake_images(index):
    fake_images = G(sample_vectors)
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    fake_fname = 'gan_mnist_images-{0:0=4d}.png'.format(index)
    print('Saving', fake_fname)
    save_image(fake_images, os.path.join(sample_dir, fake_fname), nrow=10)

In [64]:
#Define a pretty simple Generator network which has:
# 1. Linear layer which takes in an input of image_size as 64 and hidden size 256
# 2. The output of the linear layer is passed through a ReLU activation function
# 3. The result of the activation function is then passed through a Linear layer which takes in 256 input and produces 256 outputs
# 4. This result is again passed through a leaky relu layer.
# 5. Finally the output is passed through a Linear layer and the result is passed through the tanh() function.

G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [65]:
def train_generator():
    # Generate fake images and calculate loss
    z = torch.randn(batch_size, latent_size).to(device)
    fake_images = G(z)
    labels = torch.ones(batch_size, 1).to(device)
    g_loss = criterion(D(fake_images), labels)

    # Backprop and optimize
    reset_grad()
    g_loss.backward()
    g_optimizer.step()
    return g_loss, fake_images

In [ ]:
%%time

num_epochs = 300
total_step = len(train_loader)
d_losses, g_losses, real_scores, fake_scores = [], [], [], []

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        # Load a batch & transform to vectors
        images = images.reshape(batch_size, -1).to(device)
        
        # Train the discriminator and generator
        d_loss, real_score, fake_score = train_discriminator(images)
        g_loss, fake_images = train_generator()
        
        # Inspect the losses
        if (i+1) % 200 == 0:
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
            real_scores.append(real_score.mean().item())
            fake_scores.append(fake_score.mean().item())
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
        
    # Sample and save images
    save_fake_images(epoch+1)

Epoch [0/300], Step [200/600], d_loss: 2.9253, g_loss: 0.1111, D(x): 0.51, D(G(z)): 0.89
Epoch [0/300], Step [400/600], d_loss: 2.9402, g_loss: 0.1091, D(x): 0.51, D(G(z)): 0.90
Epoch [0/300], Step [600/600], d_loss: 2.9410, g_loss: 0.1087, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0001.png
Epoch [1/300], Step [200/600], d_loss: 2.9453, g_loss: 0.1086, D(x): 0.51, D(G(z)): 0.90
Epoch [1/300], Step [400/600], d_loss: 2.9478, g_loss: 0.1085, D(x): 0.51, D(G(z)): 0.90
Epoch [1/300], Step [600/600], d_loss: 2.9451, g_loss: 0.1085, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0002.png
Epoch [2/300], Step [200/600], d_loss: 2.9440, g_loss: 0.1085, D(x): 0.51, D(G(z)): 0.90
Epoch [2/300], Step [400/600], d_loss: 2.9482, g_loss: 0.1084, D(x): 0.51, D(G(z)): 0.90
Epoch [2/300], Step [600/600], d_loss: 2.9498, g_loss: 0.1084, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0003.png
Epoch [3/300], Step [200/600], d_loss: 2.9479, g_loss: 0.1084, D(x): 0.51, D(G(z)): 0.90
Epoch [3/30

Epoch [27/300], Step [400/600], d_loss: 2.9542, g_loss: 0.1083, D(x): 0.51, D(G(z)): 0.90
Epoch [27/300], Step [600/600], d_loss: 2.9456, g_loss: 0.1083, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0028.png
Epoch [28/300], Step [200/600], d_loss: 2.9473, g_loss: 0.1083, D(x): 0.51, D(G(z)): 0.90
Epoch [28/300], Step [400/600], d_loss: 2.9515, g_loss: 0.1083, D(x): 0.51, D(G(z)): 0.90
Epoch [28/300], Step [600/600], d_loss: 2.9463, g_loss: 0.1083, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0029.png
Epoch [29/300], Step [200/600], d_loss: 2.9525, g_loss: 0.1083, D(x): 0.51, D(G(z)): 0.90
Epoch [29/300], Step [400/600], d_loss: 2.9489, g_loss: 0.1083, D(x): 0.51, D(G(z)): 0.90
Epoch [29/300], Step [600/600], d_loss: 2.9486, g_loss: 0.1083, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0030.png
Epoch [30/300], Step [200/600], d_loss: 2.9483, g_loss: 0.1083, D(x): 0.51, D(G(z)): 0.90
Epoch [30/300], Step [400/600], d_loss: 2.9450, g_loss: 0.1083, D(x): 0.51, D(G(z)): 0.90
E

Epoch [54/300], Step [600/600], d_loss: 2.9516, g_loss: 0.1079, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0055.png
Epoch [55/300], Step [200/600], d_loss: 2.9535, g_loss: 0.1079, D(x): 0.51, D(G(z)): 0.90
Epoch [55/300], Step [400/600], d_loss: 2.9504, g_loss: 0.1079, D(x): 0.51, D(G(z)): 0.90
Epoch [55/300], Step [600/600], d_loss: 2.9558, g_loss: 0.1079, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0056.png
Epoch [56/300], Step [200/600], d_loss: 2.9525, g_loss: 0.1079, D(x): 0.51, D(G(z)): 0.90
Epoch [56/300], Step [400/600], d_loss: 2.9479, g_loss: 0.1079, D(x): 0.51, D(G(z)): 0.90
Epoch [56/300], Step [600/600], d_loss: 2.9541, g_loss: 0.1079, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0057.png
Epoch [57/300], Step [200/600], d_loss: 2.9527, g_loss: 0.1079, D(x): 0.51, D(G(z)): 0.90
Epoch [57/300], Step [400/600], d_loss: 2.9561, g_loss: 0.1079, D(x): 0.51, D(G(z)): 0.90
Epoch [57/300], Step [600/600], d_loss: 2.9526, g_loss: 0.1079, D(x): 0.51, D(G(z)): 0.90
S

Epoch [82/300], Step [200/600], d_loss: 2.9467, g_loss: 0.1078, D(x): 0.51, D(G(z)): 0.90
Epoch [82/300], Step [400/600], d_loss: 2.9547, g_loss: 0.1078, D(x): 0.51, D(G(z)): 0.90
Epoch [82/300], Step [600/600], d_loss: 2.9569, g_loss: 0.1078, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0083.png
Epoch [83/300], Step [200/600], d_loss: 2.9499, g_loss: 0.1078, D(x): 0.51, D(G(z)): 0.90
Epoch [83/300], Step [400/600], d_loss: 2.9545, g_loss: 0.1078, D(x): 0.51, D(G(z)): 0.90
Epoch [83/300], Step [600/600], d_loss: 2.9497, g_loss: 0.1078, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0084.png
Epoch [84/300], Step [200/600], d_loss: 2.9496, g_loss: 0.1078, D(x): 0.51, D(G(z)): 0.90
Epoch [84/300], Step [400/600], d_loss: 2.9569, g_loss: 0.1078, D(x): 0.51, D(G(z)): 0.90
Epoch [84/300], Step [600/600], d_loss: 2.9571, g_loss: 0.1078, D(x): 0.51, D(G(z)): 0.90
Saving gan_mnist_images-0085.png
Epoch [85/300], Step [200/600], d_loss: 2.9512, g_loss: 0.1078, D(x): 0.51, D(G(z)): 0.90
E